In [1]:
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [2]:
from comm_agents.data.reference_experiments import RefExperimentMass, RefExperimentCharge
from comm_agents.data.optimal_answers import get_alpha_star, get_phi_star
from ipywidgets import interact
import numpy as  np

In [3]:
# experiment parameters
GOLF_HOLE_LOC_M = .1
GOLF_HOLE_LOC_C = .1
TOLERANCE = .1
PARAM_DICT = dict(
    m=[1e-20, 1e-20],
    q=[1e-16, -1e-15],
    m_ref_m=2e-20,
    q_ref=[-1e-17, -1e-17],
    v_ref_m=2,
    m_ref_c=2e-20,
    v_ref_c=0,
    N=10,
    alpha=[0, 0],
    phi=[0, 0],
    dt=.01,
    d=.1,
    is_golf_game=False,
    gravity=True)
M_RANGES = [1e-20, 5e-20]
Q0_RANGE = [1e-16, 2e-16]
Q1_RANGE = [-1e-15, -2e-15]
V_REF_RANGE = [1, 2]

In [4]:
alpha_range_re1 = np.linspace(0, 0.6, 50)
m_range = list(np.linspace(1e-20, 1e-19, 50))
alpha_range_re2 = np.linspace(0.5, 1, 20)
q_range_re2_1 = np.linspace(1e-16, 2e-16, 50)
q_range_re2_2 = np.linspace(-1e-15, -2e-15, 50)
v_ref_range = np.linspace(1, 2, 50)
N_reange = [10, 100, 1000]

In [5]:
def get_re1_plot(alpha1, alpha2, m1, m2, v_ref, N, alpha_star):
    rem = RefExperimentMass(**PARAM_DICT)
    rem.N = N
    rem.angle = np.array([alpha1*np.pi, alpha2*np.pi])
    rem.m = np.array([m1, m2])
    rem.v_ref = v_ref
    if alpha_star:
        rem.angle, loss = get_alpha_star(rem)
        hio = rem.check_for_hole_in_one(golf_hole_loc=GOLF_HOLE_LOC_M, tolerance=TOLERANCE)
        print(f'Loss: {loss}, Hole in one: {hio}')
    rem.set_initial_state()
    rem.run()
    rem.visualize(golf_hole_loc=GOLF_HOLE_LOC_M, tolerance=TOLERANCE)
    
interact(get_re1_plot, alpha1=alpha_range_re1, alpha2=alpha_range_re1,
         m1=m_range, m2=m_range, N=N_reange, alpha_star=False, v_ref=v_ref_range)

interactive(children=(Dropdown(description='alpha1', options=(0.0, 0.012244897959183673, 0.024489795918367346,…

<function __main__.get_re1_plot(alpha1, alpha2, m1, m2, v_ref, N, alpha_star)>

In [6]:
def get_re2_plot(m1, m2, v_ref, phi1=.5, phi2=.63, q1=4e-12, q2=-3e-12, N=100, 
                 phi_star=False, y_cap=True):
    req = RefExperimentCharge(**PARAM_DICT)
    req.is_golf_game = True
    req.N = N
    req.y_cap = y_cap
    req.angle = np.array([phi1*np.pi, phi2*np.pi])
    req.q = np.array([q1, q2])
    req.m = np.array([m1, m2])
    req.v_ref = v_ref
    if phi_star:
        req.angle, loss = get_phi_star(req)
        hio = req.check_for_hole_in_one(golf_hole_loc=GOLF_HOLE_LOC_C, tolerance=TOLERANCE)
        print(f'Loss: {loss}, Hole in one: {hio}')
    req.set_initial_state()
    req.run()
    req.visualize(golf_hole_loc=GOLF_HOLE_LOC_C)
    print(req.q_ref)
interact(get_re2_plot, phi1=alpha_range_re2, phi2=alpha_range_re2,
        q1=q_range_re2_1, q2=q_range_re2_2,
         m1=m_range, m2=m_range,
         N=N_reange, phi_star=False,
         v_ref=v_ref_range,
         y_cap=True)

interactive(children=(Dropdown(description='m1', options=(1e-20, 1.183673469387755e-20, 1.36734693877551e-20, …

<function __main__.get_re2_plot(m1, m2, v_ref, phi1=0.5, phi2=0.63, q1=4e-12, q2=-3e-12, N=100, phi_star=False, y_cap=True)>

In [28]:
# some quick feasibility checks

In [7]:
from comm_agents.data.datahandler import RefExpDataset

In [9]:
ds = RefExpDataset()

In [10]:
ds[0]

(tensor([ 1.3788e-20,  3.0986e-20,  1.0992e-16, -1.4092e-15]),
 tensor([ 0.0000e+00,  2.0929e-02,  4.1859e-02,  6.2788e-02,  8.3717e-02,
          1.0465e-01,  1.2558e-01,  1.4650e-01,  1.6743e-01,  1.8836e-01,
          0.0000e+00,  1.3869e-02,  2.7739e-02,  4.1608e-02,  5.5478e-02,
          6.9348e-02,  8.3217e-02,  9.7086e-02,  1.1096e-01,  1.2483e-01,
          0.0000e+00,  3.5836e-06,  1.4335e-05,  3.2256e-05,  5.7348e-05,
          8.9617e-05,  1.2906e-04,  1.7570e-04,  2.2953e-04,  2.9055e-04,
          0.0000e+00, -2.0443e-05, -8.1755e-05, -1.8389e-04, -3.2675e-04,
         -5.1024e-04, -7.3420e-04, -9.9844e-04, -1.3028e-03, -1.6469e-03]),
 tensor([2.0000e-20, 1.7679e+00, 2.0000e-20, 1.5357e+00]),
 tensor([0.1125, 0.2692, 1.6660, 1.6690]))

In [29]:
import pandas as pd

In [30]:
df_sampels = pd.read_csv('data/reference_experiment_dat.csv')

In [31]:
df_sampels.alpha_star0.max() / np.pi

0.14264174223876083

In [32]:
df_sampels[0:1].to_dict()

{'m0': {0: 1.3787796896581534e-20},
 'm1': {0: 3.098626843255872e-20},
 'q0': {0: 1.099232064298954e-16},
 'q1': {0: -1.4092251778554867e-15},
 'o0_0_0': {0: 0.0},
 'o0_0_1': {0: 0.02092927320893955},
 'o0_0_2': {0: 0.0418585464178791},
 'o0_0_3': {0: 0.06278781962681865},
 'o0_0_4': {0: 0.0837170928357582},
 'o0_0_5': {0: 0.10464636604469776},
 'o0_0_6': {0: 0.12557563925363732},
 'o0_0_7': {0: 0.14650491246257688},
 'o0_0_8': {0: 0.16743418567151644},
 'o0_0_9': {0: 0.18836345888045603},
 'o0_1_0': {0: 0.0},
 'o0_1_1': {0: 0.01386949965385468},
 'o0_1_2': {0: 0.02773899930770936},
 'o0_1_3': {0: 0.04160849896156405},
 'o0_1_4': {0: 0.05547799861541872},
 'o0_1_5': {0: 0.0693474982692734},
 'o0_1_6': {0: 0.08321699792312809},
 'o0_1_7': {0: 0.09708649757698276},
 'o0_1_8': {0: 0.11095599723083746},
 'o0_1_9': {0: 0.12482549688469215},
 'o1_0_0': {0: 0.0},
 'o1_0_1': {0: 3.5836386088983146e-06},
 'o1_0_2': {0: 1.4335068161835401e-05},
 'o1_0_3': {0: 3.225583024265275e-05},
 'o1_0_4': {

In [15]:
1.6660089438070642/ np.pi

0.5303071172844039

In [ ]:
rem = RefExperimentMass(**PARAM_DICT)
rem.m = []